In [1]:
import os

In [2]:
%pwd

'/Users/devanshusurana/Desktop/Devanshu/College/Capstone Project/Chest-Cancer-Prediction-using-ML-Flow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/devanshusurana/Desktop/Devanshu/College/Capstone Project/Chest-Cancer-Prediction-using-ML-Flow-DVC'

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

In [10]:
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [15]:
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e


[2025-05-05 20:25:08,638: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-05 20:25:08,643: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-05 20:25:08,645: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 2s 223ms/step - loss: 0.6704 - accuracy: 0.4902
[2025-05-05 20:25:10,989: INFO: common: json file saved at: scores.json]


2025/05/05 20:25:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/nl/tnp27lsx4r543mszgcbml8vm0000gn/T/tmpvzrxppqy/model/data/model/assets
[2025-05-05 20:25:15,705: INFO: builder_impl: Assets written to: /var/folders/nl/tnp27lsx4r543mszgcbml8vm0000gn/T/tmpvzrxppqy/model/data/model/assets]
An unexpected error occurred: module 'tensorflow.keras' has no attribute '__version__'


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

[2025-01-27 22:33:33,801:INFO:common:yaml file: config/config.yaml loaded successfully]
[2025-01-27 22:33:33,802:INFO:common:yaml file: params.yaml loaded successfully]
[2025-01-27 22:33:33,803:INFO:common:created directory at: artifacts]
An error occurred: No file or directory found at artifacts/training/model.h5


In [20]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
tensorflow.keras.__version__ = tf.__version__

model = tf.keras.models.load_model("artifacts/training/model.h5")

from dataclasses import dataclass
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
# %pip install mlflow tensorflow~=2.13.0
# %pip install keras~=2.13.0

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        
        # Predictions
        y_pred_probs = self.model.predict(self.valid_generator)
        y_pred = y_pred_probs.argmax(axis=1)
        y_true = self.valid_generator.classes

        # Calculate precision, recall, and F1-score
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')

        # Compute specificity
        cm = confusion_matrix(y_true, y_pred)
        tn = cm[0, 0]  # True negatives
        fp = cm[0, 1]  # False positives
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        self.performance_scores = {
            "loss": self.score[0],
            "accuracy": self.score[1],
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "specificity": specificity
        }
        print("Metric           Value")
        print("-----------------------------")
        for metric, value in self.performance_scores.items():
            print(f"{metric:<17} {value}")

        self.save_score()

    def save_score(self):
        print(f"Performance Scores: {self.performance_scores}")

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(self.performance_scores)

            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools
                else:
                    raise e

# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e

[2025-05-05 20:27:36,044: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-05 20:27:36,046: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-05 20:27:36,047: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 2s 223ms/step
Metric           Value
-----------------------------
loss              0.6704476475715637
accuracy          0.4901960790157318
precision         0.4943000455996352
recall            0.49019607843137253
f1_score          0.422722029988466
specificity       0.15384615384615385
Performance Scores: {'loss': 0.6704476475715637, 'accuracy': 0.4901960790157318, 'precision': 0.4943000455996352, 'recall': 0.49019607843137253, 'f1_score': 0.422722029988466, 'specificity': 0.15384615384615385}


2025/05/05 20:27:41 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/nl/tnp27lsx4r543mszgcbml8vm0000gn/T/tmppjrjub3u/model/data/model/assets
[2025-05-05 20:27:44,881: INFO: builder_impl: Assets written to: /var/folders/nl/tnp27lsx4r543mszgcbml8vm0000gn/T/tmppjrjub3u/model/data/model/assets]


2025/05/05 20:27:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/nl/tnp27lsx4r543mszgcbml8vm0000gn/T/tmppjrjub3u/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
/opt/anaconda3/envs/cancer-1/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/05/05 20:29:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 12
Created version '12' of model 'VGG16Model'.


In [15]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

#     def _valid_generator(self):
#         datagenerator_kwargs = dict(
#             rescale=1.0 / 255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         # Fixed `model` to `self.model`
#         self.score = self.model.evaluate(self.valid_generator)
#         self.save_score()

#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)

#     def log_into_mlflow(self):
#         mlflow.set_registry_uri(self.config.mlflow_uri)
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#         with mlflow.start_run():
#             mlflow.log_params(self.config.all_params)
#             mlflow.log_metrics(
#                 {"loss": self.score[0], "accuracy": self.score[1]}
#             )
#             # Model registry does not work with file store
#             if tracking_url_type_store != "file":
#                 # Register the model
#                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
#             else:
#                 mlflow.keras.log_model(self.model, "model")


In [ ]:
# try:
#     config = ConfigurationManager()
#     eval_config = config.get_evaluation_config()
#     evaluation = Evaluation(eval_config)
#     evaluation.evaluation()
#     evaluation.log_into_mlflow()

# except Exception as e:
#    raise e

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode='categorical',  # Explicitly set class_mode  # Explicitly define class order
            **dataflow_kwargs
        )
        
        # Print class indices to verify the order
        print("Class indices:", self.valid_generator.class_indices)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e
